In [2]:
import string
import re 
import pandas as pd
import numpy as np
import fuzzywuzzy
import thefuzz
from thefuzz import process
from fuzzywuzzy import fuzz
import pickle
from csv import DictWriter

def readfile(filename):
    df = pd.read_csv('/Users/samanthalee/Desktop/csv/' + filename +'.csv')
    return df

def check(df):
    print(df.head())
    print(df.dtypes)
    
#function for cleaning zip col from test file 
def clean(df):
    df.zip = df.zip.astype(str) #turning zip into string
    for ind in df.index: 
        df.zip[ind] = df.zip[ind].split('.', 1)[0] #remove .0 after zip 
        df.zip[ind] = df.zip[ind].strip(' ') #remove spaces before and after 

#function for replacing address with close match
def replace_with_fuzzy_match(test, standard, threshold):
    test_cleaned = [x for x in test["address"] if not(pd.isnull(x))]
    standard_cleaned = [x for x in standard["address"].unique() if not(pd.isnull(x))]
    tuples_list = [max([(fuzz.ratio(i,j),j) for j in standard_cleaned]) for i in test_cleaned]
    similarity_score, fuzzy_match = map(list,zip(*tuples_list))
    df = pd.DataFrame({"list_A":test_cleaned, "fuzzy match": fuzzy_match, "similarity score": similarity_score})
    for ind in test.index:
        if df['similarity score'][ind] >= threshold:
            test['address'][ind] = df['fuzzy match'][ind]
    return test

#function for mapping dictionary to missing values 
def map_to_dict(test, column):
    with open('/Users/samanthalee/Desktop/' + column +'.pkl', 'rb') as fp:
        x = pickle.load(fp)
        test[column] = test[column].fillna(test['address'].map(x))

#function for combining address, state, city, zip col in test datasets for matching      
def combine_test(df):
    df_combine = df.address + df.state + df.city + df.zip 
    df_combine = df_combine.dropna()
    df_combine = df_combine.to_frame()
    return df_combine

#function for calculating coverage ratios
def merge_and_calculate(test_combine, standard_combine):
    join = pd.merge(standard_combine, test_combine, how = 'inner') 
    coverage = join.shape[0]/original_row 
    return join.shape[0],coverage 

#read files
df_test = readfile('test')
df_all = readfile('all_addresses')

#calculate deniminator for coverage ratio 
original_row = df_test.shape[0]

#creating test file copy for calculating previous coverage ratio 
df_test2 = df_test.copy()
clean(df_test2)

#clean test dataset
for ind in df_test.index: 
    result = df_test.address[ind].split(',', 1)[0] 
    df_test.address[ind] = result
df_test['state'] = df_test['state'].str.upper() #turning all state into uppercase
df_test = df_test.drop(df_test[df_test["address"].str.contains("#NAM")].index) 
#drop rows where address contains 
#NAM since they can never be matched 
df_test = df_test.reset_index()

#clean standard dataset
df_all = df_all.dropna() #remove NaN rows 
df_all = df_all[df_all.address != '#NAME?'] #remove rows containing #NAME?

#replacing test file with close match using fuzzywuzzy
#setting threshold as 80%, i.e. only matches > 80% is considered as wrong entry, < 80% is considered as a different entry
df_test = replace_with_fuzzy_match(df_test, df_all, 80)

#using dictionary created from standard datasets to map missing values in zip, state and city columns         
map_to_dict(df_test, 'zip')
map_to_dict(df_test, 'state')
map_to_dict(df_test, 'city')

#clean new test file for merging 
clean(df_test)

#combining address, state, city, zip col in standard dataset for matching 
standard_combine = df_all.address + df_all.state + df_all.city + df_all.zip 
standard_combine = standard_combine.drop_duplicates() 
standard_combine = standard_combine.to_frame()

#combining address, state, city, zip col in test dataset for matching 
test_combine = combine_test(df_test)
test2_combine = combine_test(df_test2)

#calculating coverage ratio 
print('new coverage: ', merge_and_calculate(test_combine, standard_combine)[1])
print('prior coverage: ', merge_and_calculate(test2_combine, standard_combine)[1])
print(df_test)

#appending merged row into one csv file
field_names = ['coverage']
dict = {'coverage': merge_and_calculate(test_combine, standard_combine)[0]}

with open('/Users/samanthalee/Desktop/total.csv', 'a') as f_object:
    dictwriter_object = DictWriter(f_object, fieldnames=field_names)
    dictwriter_object.writerow(dict)
    f_object.close()


/var/folders/9z/0pvw3fjs5bj17s3vcxnxzn1r0000gn/T/ipykernel_1427/3162532924.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.zip[ind] = df.zip[ind].split('.', 1)[0] #remove .0 after zip
/var/folders/9z/0pvw3fjs5bj17s3vcxnxzn1r0000gn/T/ipykernel_1427/3162532924.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.zip[ind] = df.zip[ind].strip(' ') #remove spaces before and after
/var/folders/9z/0pvw3fjs5bj17s3vcxnxzn1r0000gn/T/ipykernel_1427/3162532924.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

new coverage:  1.0
prior coverage:  0.36666666666666664
    index  uid                    address           city state    zip
0       0    1         1114 Mt George Ave           Napa    CA  94558
1       1    2         5429 W Slauson Ave    Los Angeles    CA  90056
2       2    3              1213 Court St        Alameda    CA  94501
3       3    4            7707 Jayseel St    Los Angeles    CA  91042
4       4    5           12273 Mulhall St       El Monte    CA  91732
5       5    6                 17 21st St     Greenville    PA  16125
6       6   10              1699 Lunt Ave    Des Plaines    IL  60018
7       7   11              631 N 21st St       San Jose    CA  95112
8       8   12           18309 3rd Ave NW      Shoreline    WA  98177
9       9   15         1608 N Comstock Ct        Visalia    CA  93292
10     10   16            6170 Alvord Way     Pleasanton    CA  94588
11     11   17           2670 BERNWOOD ST         Duarte    CA  91010
12     12   19             130 Par